## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [1]:
import datetime
import os

from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware
import pandas as pd
from notebooks import get_twap_bpt_price
from notebooks import get_block_by_ts
from notebooks.constants import ARB_CORE_POOLS
from notebooks.constants import MAINNET_CORE_POOLS
from notebooks.constants import POLYGON_CORE_POOLS
from notebooks.constants import BASE_CORE_POOLS

load_dotenv()
arb_web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
eth_web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))
base_web3 = Web3(Web3.HTTPProvider(os.environ["BASENODEURL"]))
poly_web3 = Web3(Web3.HTTPProvider("https://polygon-rpc.com"))
poly_web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# ARBITRUM
# arb_block_now = arb_web3.eth.block_number - 1000
timestamp_now = 1694750400
arb_block = get_block_by_ts(timestamp_now, "arbitrum")  # 18 August 2023
# Given Arb block time, we want to look back 2 weeks:
arb_timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
arb_block_2_weeks_ago = get_block_by_ts(arb_timestamp_2_weeks_ago, "arbitrum")
# Convert to datetime:
arb_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
arb_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(arb_timestamp_2_weeks_ago)
# MAINNET
mainnet_block = get_block_by_ts(timestamp_now, "mainnet")
# Given mainnet block time, we want to look back 2 weeks:
mainnet_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
mainnet_block_2_weeks_ago = get_block_by_ts(mainnet_timestamp_2_weeks_ago, "mainnet")
# Convert to datetime:
mainnet_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
mainnet_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(mainnet_timestamp_2_weeks_ago)

# BASE
base_block = get_block_by_ts(timestamp_now, "base")
# Given base block time, we want to look back 2 weeks:
base_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
base_block_2_weeks_ago = get_block_by_ts(base_timestamp_2_weeks_ago, "base")
# Convert to datetime:
base_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
base_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(base_timestamp_2_weeks_ago)

# POLYGON
poly_block = get_block_by_ts(timestamp_now, "polygon")
# Given polygon block time, we want to look back 2 weeks:
poly_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
poly_block_2_weeks_ago = get_block_by_ts(poly_timestamp_2_weeks_ago, "polygon")
# Convert to datetime:
poly_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
poly_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(poly_timestamp_2_weeks_ago)

print("Collecting bpt prices for Base")
base_bpt_twap_prices = {}
for base_pool in BASE_CORE_POOLS:
    base_bpt_twap_prices[base_pool] = get_twap_bpt_price(base_pool, 'base', base_web3,
                                                         start_date=base_datetime_now.date(), block_number=base_block)
print("Collecting bpt prices for Arbitrum")
arb_bpt_twap_prices = {}
for arb_pool in ARB_CORE_POOLS:
    arb_bpt_twap_prices[arb_pool] = get_twap_bpt_price(arb_pool, 'arbitrum', arb_web3, start_date=arb_datetime_now.date(),
                                                       block_number=arb_block)

print("Collecting bpt prices for Mainnet")
mainnet_bpt_twap_prices = {}
for mainnet_pool in MAINNET_CORE_POOLS:
    mainnet_bpt_twap_prices[mainnet_pool] = get_twap_bpt_price(mainnet_pool, 'mainnet', eth_web3,
                                                               start_date=mainnet_datetime_now.date(),
                                                               block_number=mainnet_block)

print("Collecting bpt prices for Polygon")
polygon_bpt_twap_prices = {}
for polygon_pool in POLYGON_CORE_POOLS:
    polygon_bpt_twap_prices[polygon_pool] = get_twap_bpt_price(polygon_pool, 'polygon', poly_web3,
                                                               start_date=poly_datetime_now.date(), block_number=poly_block)

# Convert to dataframe and print, merge all three dataframes
arb_bpt_twap_prices_df = pd.DataFrame.from_dict(arb_bpt_twap_prices, orient='index')
mainnet_bpt_twap_prices_df = pd.DataFrame.from_dict(mainnet_bpt_twap_prices, orient='index')
polygon_bpt_twap_prices_df = pd.DataFrame.from_dict(polygon_bpt_twap_prices, orient='index')
base_bpt_twap_prices_df = pd.DataFrame.from_dict(base_bpt_twap_prices, orient='index')

In [2]:
# Arbitrum TWAP prices:
arb_bpt_twap_prices_df

,0
0xade4a71bb62bec25154cfc7e6ff49a513b491e81000000000000000000000497,1624.824663978241822195294531
0x9791d590788598535278552eecd4b211bfc790cb000000000000000000000498,1625.344235183566077290910293
0x4a2f6ae7f3e5d715689530873ec35593dc28951b000000000000000000000481,1629.419505393894093241957607
0x423a1323c871abc9d89eb06855bf5347048fc4a5000000000000000000000496,1.000280966766936922825826068
0x32df62dc3aed2cd6224193052ce665dc181658410002000000000000000003bd,1.089197381211832968347552422
0x0c8972437a38b389ec83d1e666b69b8a4fcf8bfd00000000000000000000049e,1624.004796170085640502940404


In [3]:
# Mainnet TWAP prices:
mainnet_bpt_twap_prices_df

,0
0x93d199263632a4ef4bb438f1feb99e57b4b5f0bd0000000000000000000005c2,1624.671669576571282985296997
0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112,1665.092865532654569304375932
0xe7e2c68d3b13d905bbb636709cf4dfd21076b9d20000000000000000000005ca,1609.505529342165392506361900
0xf7a826d47c8e02835d94fb0aa40f0cc9505cb1340002000000000000000005e0,1624.040830847811722959043062
0xf16aee6a71af1a9bc8f56975a4c2705ca7a782bc0002000000000000000004bb,24.93213796900873485293154129
0xb08885e6026bab4333a80024ec25a1a3e1ff2b8a000200000000000000000445,1637.374646097921670227738398
0x8353157092ed8be69a9df8f95af097bbf33cb2af0000000000000000000005d9,0.9837261409796526292723900507
0xdfe6e7e18f6cc65fa13c8d8966013d4fda74b6ba000000000000000000000558,1632.951961172472323274683402
0x5f1f4e50ba51d723f12385a8a9606afc3a0555f5000200000000000000000465,53.06393404391770336711065633
0x1ee442b5326009bb18f2f472d3e0061513d1a0ff000200000000000000000464,59.00866122834476785368647987


In [4]:
# Polygon TWAP prices:
polygon_bpt_twap_prices_df

,0
0xf0ad209e2e969eaaa8c882aac71f02d8a047d5c2000200000000000000000b49,0.5414459234689891732317339027
0xee278d943584dd8640eaf4cc6c7a5c80c0073e85000200000000000000000bc7,0.5393899124051555957882508035
0x89b753153678bc434c610b7e9182297cada8ff29000000000000000000000c21,0.5369363464436515674461473867
0xcd78a20c597e367a4e478a2411ceb790604d7c8f000000000000000000000c22,0.5370201093525471584545017470
0xdc31233e09f3bf5bfe5c10da2014677c23b6894c000000000000000000000c23,1623.902166775320147917739091


In [5]:
# Base TWAP prices:
base_bpt_twap_prices_df

,0
0xfb4c2e6e6e27b5b4a07a36360c89ede29bb3c9b6000000000000000000000026,1624.588029734466136359339174
0xc771c1a5905420daec317b154eb13e4198ba97d0000000000000000000000023,1627.903553206134066310896620
0x0c659734f1eef9c63b7ebdf78a164cdd745586db000000000000000000000046,1.000328667223006881844029683


In [6]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
    where: {{ protocolFee_not: null }}
  ) {{
    pool {{
      address
      id
      symbol
      totalProtocolFeePaidInBPT
      tokens {{
        symbol
        address
        paidProtocolFees
      }}
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

## Fetching data from the Balancer subgraphs

In [7]:
from notebooks.constants import BASE_BALANCER_GRAPH_URL
from notebooks.constants import POLYGON_BALANCER_GRAPH_URL
from notebooks.constants import ARB_BALANCER_GRAPH_URL
from notebooks.constants import MAINNET_BALANCER_GRAPH_URL
from typing import Dict
from typing import List
from typing import Optional

from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport


# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots(block: int, graph_url: str) -> Optional[List[Dict]]:
    client = make_gql_client(graph_url)
    all_pools = []
    limit = 1000
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, block=block)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if offset >= 5000:
            break
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


arbi_pool_snapshots_now = get_balancer_pool_snapshots(arb_block, ARB_BALANCER_GRAPH_URL)
arbi_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(arb_block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

mainnet_pool_snapshots_now = get_balancer_pool_snapshots(mainnet_block, MAINNET_BALANCER_GRAPH_URL)
mainnet_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(mainnet_block_2_weeks_ago),
                                                                 MAINNET_BALANCER_GRAPH_URL)

polygon_pool_snapshots_now = get_balancer_pool_snapshots(poly_block, POLYGON_BALANCER_GRAPH_URL)
polygon_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(poly_block_2_weeks_ago),
                                                                 POLYGON_BALANCER_GRAPH_URL)

base_pool_snapshots_now = get_balancer_pool_snapshots(base_block, BASE_BALANCER_GRAPH_URL)
base_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(base_block_2_weeks_ago),
                                                              BASE_BALANCER_GRAPH_URL)

## Extract fee data for CORE pools:


In [12]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: str, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.date) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Calculate fees
        pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
        pool_swap_fee = float(pool_snapshot_now['swapFees']) - float(pool_snapshot_2_weeks_ago['swapFees'])
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = arb_bpt_twap_prices[pool] if chain == 'arbitrum' else mainnet_bpt_twap_prices[
            pool] if chain == 'mainnet' else polygon_bpt_twap_prices[pool] if chain == 'polygon' else base_bpt_twap_prices[pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['symbol']].append({
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': arb_datetime_2_weeks_ago,
                'time_to': arb_datetime_now,
                'chain': chain,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain, start_date) or 0
                token_fees[pool_snapshot_now['pool']['symbol']].append({
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': arb_datetime_2_weeks_ago,
                    'time_to': arb_datetime_now,
                    'chain': chain,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['symbol']] = {
            'pool_fee': round(pool_fee, 2),
            'swap_fee': round(pool_swap_fee, 2),
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                      token_fees[pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': arb_datetime_2_weeks_ago,
            'time_to': arb_datetime_now,
            'chain': chain,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']],
        }
    return fees


arb_fees = collect_fee_info(ARB_CORE_POOLS, 'arbitrum', arbi_pool_snapshots_now,
                            arbi_pool_snapshots_2_weeks_ago, arb_datetime_now.date())
mainnet_fees = collect_fee_info(MAINNET_CORE_POOLS, 'mainnet', mainnet_pool_snapshots_now,
                                mainnet_pool_snapshots_2_weeks_ago, mainnet_datetime_now.date())
polygon_fees = collect_fee_info(POLYGON_CORE_POOLS, 'polygon', polygon_pool_snapshots_now,
                                polygon_pool_snapshots_2_weeks_ago, poly_datetime_now.date())
base_fees = collect_fee_info(BASE_CORE_POOLS, 'base', base_pool_snapshots_now, base_pool_snapshots_2_weeks_ago,
                             base_datetime_now.date())
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**arb_fees, **mainnet_fees, **polygon_fees, **base_fees}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [11]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain', 'pool_fee'], ascending=False)

,pool_fee,swap_fee,bpt_token_fee,bpt_token_fee_in_usd,token_fees_in_usd,time_from,time_to,chain
maticX-WMATIC-BPT,1572.58,322.17,2946.70,1582.44,0,2023-09-01 07:00:00,2023-09-15 07:00:00,polygon
ECLP-WMATIC-stMATIC,1480.21,279.86,2745.75,1486.68,0,2023-09-01 07:00:00,2023-09-15 07:00:00,polygon
ECLP-WMATIC-MATICX,307.23,70.58,570.61,307.78,0,2023-09-01 07:00:00,2023-09-15 07:00:00,polygon
stMATIC-WMATIC-BPT,2.58,1.42,4.62,2.48,0,2023-09-01 07:00:00,2023-09-15 07:00:00,polygon
wstETH-WETH-BPT,0.00,0.00,0.00,0.00,0.00,2023-09-01 07:00:00,2023-09-15 07:00:00,polygon
B-rETH-STABLE,15033.43,2024.52,0.00,0.00,14944.71,2023-09-01 07:00:00,2023-09-15 07:00:00,mainnet
50rETH-50BADGER,8173.75,2041.90,137.83,8133.10,0,2023-09-01 07:00:00,2023-09-15 07:00:00,mainnet
swETH-WETH-BPT,4990.34,634.77,3.11,5002.92,0,2023-09-01 07:00:00,2023-09-15 07:00:00,mainnet
20WETH-80ALCX,4766.92,4542.19,171.57,4277.72,0,2023-09-01 07:00:00,2023-09-15 07:00:00,mainnet
50rETH-50RPL,1986.93,1170.02,9.78,1952.83,0,2023-09-01 07:00:00,2023-09-15 07:00:00,mainnet


## Now let's calculate bribes paid to the pools

In [13]:
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(eth_web3, mainnet_block)

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.
FEE = Decimal(0.5)  # 50% goes to fees
# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str) -> Dict[str, Dict]:
    pool_bribs = {}
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "chain": chain,
            "aura_bribes": pool_fees * FEE * aura_vebal_share,
            "bal_bribes": pool_fees * FEE * (1 - aura_vebal_share),
            "pool_total": pool_fees,
            "fees_taken": pool_fees * FEE,
        }
    return pool_bribs
mainnet_bribes = calc_and_split_bribes(mainnet_fees, 'mainnet')
arb_bribes = calc_and_split_bribes(arb_fees, 'arbitrum')
polygon_bribes = calc_and_split_bribes(polygon_fees, 'polygon')
base_bribes = calc_and_split_bribes(base_fees, 'base')
# Convert to dataframe
joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes, **base_bribes}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'../data/bribs/bribes_{arb_datetime_2_weeks_ago.date()}_{arb_datetime_now.date()}.csv')
joint_bribes_df

,chain,aura_bribes,bal_bribes,pool_total,fees_taken
wstETH/rETH/cbETH,arbitrum,370.3736070809998385118329073,562.0163929190001614881670927,1864.78,932.390
4POOL-BPT,arbitrum,99.99479628749781673825727190,151.7352037125021832617427281,503.46,251.730
RDNT-WETH,arbitrum,2251.030912183438367932337752,3415.784087816561632067662248,11333.63,5666.815
wstETH/rETH/sfrxETH,arbitrum,0.003972303511202392116086969050,0.006027696488797607883913030950,0.02,0.010
rETH-WETH-BPT,base,0E-31,0E-31,0.00,0.000
cbETH/WETH,base,369.2712928566411746996187733,560.3437071433588253003812267,1859.23,929.615
B-rETH-STABLE,mainnet,2968.246200345075074060304362,4504.108799654924925939695638,14944.71,7472.355
swETH-WETH-BPT,mainnet,993.6558341132335782706909600,1507.804165886766421729309040,5002.92,2501.460
ECLP-wstETH-cbETH,mainnet,0E-31,0E-31,0.00,0.000
20WETH-80ALCX,mainnet,849.6201087970348401413774622,1289.239891202965159858622538,4277.72,2138.860
